<h1>Extracting and Visualizing Stock Data</h1>
<h2>Description</h2>


In this notebook we extract stock data using <code>yfinance</code> and also by means of webscraping using <code>BeautifulSoup</code>. We will cast the extracted data into Pandas dataframes and then use a function to visualize the data.

<h2>Table of Contents</h2>
<div class="alert alert-block alert-info" style="margin-top: 20px">
    <ul>
        <li>Define a Function for visualization</li>
        <li>Use yfinance to Extract Tesla and GameStop Stock Data</li>
        <li>Use Webscraping to Extract Tesla and GameStop Revenue Data</li>
        <li>Plot Tesla and GameStop Stock Graphs</li>
    </ul>
</div>
<hr>

Run the next cell to install the right packages and versions if needed.

In [86]:
#!pip install yfinance==0.1.70
#!pip3 install --upgrade nbformat

Next, we import all the necessary libraries.

In [87]:
import yfinance as yf
import pandas as pd
import requests
from bs4 import BeautifulSoup
import plotly.graph_objects as go
from plotly.subplots import make_subplots

## Define Graphing Function


In this section, we define the function `make_graph`. It takes a dataframe with stock data (dataframe must contain Date and Close columns), a dataframe with revenue data (dataframe must contain Date and Revenue columns), and the name of the stock.

In [102]:
def make_graph(stock_data, revenue_data, stock):
    fig = make_subplots(rows=2, cols=1, shared_xaxes=True, subplot_titles=("Historical Share Price", "Historical Revenue"), vertical_spacing = .3)
    stock_data_specific = stock_data[stock_data.Date <= '2021--01-01']
    revenue_data_specific = revenue_data[revenue_data.Date <= '2021--01-01']
    fig.add_trace(go.Scatter(x=pd.to_datetime(stock_data_specific.Date, infer_datetime_format=True), y=stock_data_specific.Close.astype("float"), name="Share Price"), row=1, col=1)
    fig.add_trace(go.Scatter(x=pd.to_datetime(revenue_data_specific.Date, infer_datetime_format=True), y=revenue_data_specific.Revenue.astype("float"), name="Revenue"), row=2, col=1)
    fig.update_xaxes(title_text="Date", row=1, col=1)
    fig.update_xaxes(title_text="Date", row=2, col=1)
    fig.update_yaxes(title_text="Price ($US)", row=1, col=1)
    fig.update_yaxes(title_text="Revenue ($US Millions)", row=2, col=1)
    fig.update_layout(showlegend=False,
    height=800,
    title=stock,
    xaxis_rangeslider_visible=True)
    fig.show()

## Using yfinance to Extract Stock Data


Using the `Ticker` function we enter the ticker symbol of the stock we want to extract data on to create a ticker object. The stock is Tesla and its ticker symbol is `TSLA`.


In [89]:
tesla = yf.Ticker("TSLA")

Using the function `history` we can extract stock information and save it in a dataframe named `tesla_data`. Setting the `period` parameter to `max` allow us to get information for the maximum amount of time.


In [90]:
tesla_data = tesla.history(period='max')

Next, we **reset the index** using the `reset_index(inplace=True)` function on the tesla_data DataFrame, since it will use the first column as index by default.

In [91]:
tesla_data.reset_index(inplace=True)
tesla_data.head()

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits
0,2010-06-29,1.266667,1.666667,1.169333,1.592667,281494500,0,0.0
1,2010-06-30,1.719333,2.028000,1.553333,1.588667,257806500,0,0.0
2,2010-07-01,1.666667,1.728000,1.351333,1.464000,123282000,0,0.0
3,2010-07-02,1.533333,1.540000,1.247333,1.280000,77097000,0,0.0
4,2010-07-06,1.333333,1.333333,1.055333,1.074000,103003500,0,0.0


We now do the same for GameStop, which ticker symbol is `GME`.

In [92]:
GameStop = yf.Ticker('GME')
gme_data = GameStop.history(period='max')
gme_data.reset_index(inplace=True)
gme_data.head()

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits
0,2002-02-13,1.620128,1.693350,1.603296,1.691666,76216000,0.0,0.0
1,2002-02-14,1.712707,1.716074,1.670626,1.683251,11021600,0.0,0.0
2,2002-02-15,1.683250,1.687458,1.658001,1.674834,8389600,0.0,0.0
3,2002-02-19,1.666418,1.666418,1.578047,1.607504,7410400,0.0,0.0
4,2002-02-20,1.615920,1.662210,1.603296,1.662210,6892800,0.0,0.0


## Webscraping to Extract Revenue Data


We use the `requests` library to download the webpage https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0220EN-SkillsNetwork/labs/project/revenue.htm and save the text of the response as a variable named `html_data`.

In [93]:
url = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0220EN-SkillsNetwork/labs/project/revenue.htm"
html_data = requests.get(url).text

We can parse the html data using `BeautifulSoup`.


In [94]:
soup = BeautifulSoup(html_data, 'html5lib')

Using the `read_html` function we extract the table with `Tesla Quarterly Revenue` (it is the second table so it's index is 1) and store it into a dataframe named `tesla_revenue`. The dataframe should have columns `Date` and `Revenue`.

In [95]:
tesla_revenue = pd.read_html(str(soup))[1]
tesla_revenue.columns=['Date', 'Revenue']
tesla_revenue.head()

,Date,Revenue
0,2022-09-30,"$21,454"
1,2022-06-30,"$16,934"
2,2022-03-31,"$18,756"
3,2021-12-31,"$17,719"
4,2021-09-30,"$13,757"


Let us remove the comma and dollar sign from the `Revenue` column. We also need to remove entries that are NaN or empty.

In [96]:
tesla_revenue["Revenue"] = tesla_revenue['Revenue'].str.replace(',',"")
tesla_revenue["Revenue"] = tesla_revenue['Revenue'].str.replace('$',"")
tesla_revenue.dropna(inplace=True)
tesla_revenue = tesla_revenue[tesla_revenue['Revenue'] != ""]

tesla_revenue.head()

,Date,Revenue
0,2022-09-30,21454
1,2022-06-30,16934
2,2022-03-31,18756
3,2021-12-31,17719
4,2021-09-30,13757


We can now do the same with GameStop by extracting the information on the webpage https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0220EN-SkillsNetwork/labs/project/stock.html.

In [97]:
url2 = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0220EN-SkillsNetwork/labs/project/stock.html'
html_data2 = requests.get(url2).text

Parse the html data using `BeautifulSoup`.


In [98]:
soup2 = BeautifulSoup(html_data2, 'html5lib')

Let us make the same transformations we did for the Tesla data.

In [99]:
gme_revenue = pd.read_html(str(soup2))[1]
gme_revenue.columns = ['Date', 'Revenue']
gme_revenue['Revenue'] = gme_revenue['Revenue'].str.replace('$','')
gme_revenue['Revenue'] = gme_revenue['Revenue'].str.replace(',','')
gme_revenue.dropna(inplace=True)
gme_revenue = gme_revenue[gme_revenue['Revenue'] != ""]

gme_revenue.head()

,Date,Revenue
0,2020-04-30,1021
1,2020-01-31,2194
2,2019-10-31,1439
3,2019-07-31,1286
4,2019-04-30,1548


## Plot Tesla and GameStop Stock Graphs

We are ready to use the `make_graph` function to graph the Tesla Stock Data.

In [103]:
make_graph(tesla_data, tesla_revenue, 'Tesla')

C:\Users\javiv\AppData\Local\Temp\ipykernel_14652\1266502693.py:5: UserWarning:

The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.

C:\Users\javiv\AppData\Local\Temp\ipykernel_14652\1266502693.py:6: UserWarning:

The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.



We can graph the GameStop Stock Data in a similar way.

In [104]:
make_graph(gme_data, gme_revenue, 'GameStop')

C:\Users\javiv\AppData\Local\Temp\ipykernel_14652\1266502693.py:5: UserWarning:

The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.

C:\Users\javiv\AppData\Local\Temp\ipykernel_14652\1266502693.py:6: UserWarning:

The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.



<h2>Conclusion:</h2> 

We have successfully extracted stock data using `yfinance` and webscraping with `BeautifulSoup`. Furthermore, we were able to store the data into a Pandas dataframe and used a function to visualize the data.
